In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cuda


In [2]:
with open('wizard-of-oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)

vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [15]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [8]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)


print('targets:')
print(y)

tensor([ 6251, 46257, 80641, 42710])
inputs:
tensor([[54,  1, 69, 54, 71, 54, 72, 68],
        [73, 72,  1, 55, 58, 73, 73, 58],
        [65, 68, 71, 58, 57,  1, 72, 74],
        [ 1, 76, 54, 73, 56, 61, 58, 57]], device='cuda:0')
targets:
tensor([[ 1, 69, 54, 71, 54, 72, 68, 65],
        [72,  1, 55, 58, 73, 73, 58, 71],
        [68, 71, 58, 57,  1, 72, 74, 67],
        [76, 54, 73, 56, 61, 58, 57,  1]], device='cuda:0')


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [14]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


w-VxIz)kjhb9AJu:vCEI(DBs00﻿hr:hqNbfhqg[HMhApZd[kl&M﻿]3itq-*nj;Zw_D﻿w5u5'IGDYEP[:Fy2"Mqt*VWYxopwxkV[HhFy
LUrSrOzM_3sb?w;vzTuP])gg'hv5fNhggI]]LF.Dcv0OVApUUBPVvBSVV_8TNuVAACH(.oiJuC2's0ggMK
rjr5X
CFC74Iv﻿YbNzM_)d-EJ:X
DD-Goa﻿9PA,FVU]3]oK﻿Ppe!,-Ib' YUWu4 u2UWL4OUhn3C*!-!,[]sAn[o?,H"P2fpKuunWVTNuhWh3bu gDKiPV(N9"lHaggyZfF0!-BMXC,blHKOwqtOwI()S3r
vx[NDS
i
f6146
mJewiH;ibV2kOU7;f:m*!:ZNWlHKDw
'XeGur﻿3g[WQOU&*V[kjWqxY!1U-*&
,fa4oTNI(Nn:Db.s,4R[ky)q0OJ5wI(O-VaSXh_VO,l4 g_K"QyVU0Lkg5XN]stEkZnt!X:﻿bT2Nf]_R
